# Exercice 2: Classification system with KNN - To Loan or Not To Loan
by Jean Nanchen and Aurélien Héritier

## Imports

Import some useful libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split

## a. Getting started

### Data loading

The original dataset comes from the Kaggle's [Loan Prediction](https://www.kaggle.com/ninzaami/loan-predication) problem. The provided dataset has already undergone some processing, such as removing some columns and invalid data. Pandas is used to read the CSV file.

In [2]:
data = pd.read_csv("loandata.csv")

Display the head of the data.

In [3]:
data.head()

Gender Married     Education  TotalIncome  LoanAmount  CreditHistory  \
0   Male     Yes      Graduate       6091.0       128.0            1.0   
1   Male     Yes      Graduate       3000.0        66.0            1.0   
2   Male     Yes  Not Graduate       4941.0       120.0            1.0   
3   Male      No      Graduate       6000.0       141.0            1.0   
4   Male     Yes      Graduate       9613.0       267.0            1.0   

  LoanStatus  
0          N  
1          Y  
2          Y  
3          Y  
4          Y

Data's columns:
* **Gender:** Applicant gender (Male/ Female)
* **Married:** Is the Applicant married? (Y/N)
* **Education:** Applicant Education (Graduate/ Not Graduate)
* **TotalIncome:** Applicant total income (sum of `ApplicantIncome` and `CoapplicantIncome` columns in the original dataset)
* **LoanAmount:** Loan amount in thousands
* **CreditHistory:** Credit history meets guidelines
* **LoanStatus** (Target)**:** Loan approved (Y/N)

### Data preprocessing

Define a list of categorical columns to encode.

In [4]:
categorical_columns = ["Gender", "Married", "Education", "LoanStatus"]

Encode categorical columns using the [`OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) of scikit learn.

In [5]:
data[categorical_columns] = OrdinalEncoder().fit_transform(data[categorical_columns])

Split into `X` and `y`.

In [6]:
X = data.drop(columns="LoanStatus")
y = data.LoanStatus

Normalize data using the [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) of scikit learn.

In [7]:
X[X.columns] = StandardScaler().fit_transform(X[X.columns])

Convert `y` type to `int` 

In [8]:
y = y.astype(int)

Split dataset into train and test sets.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

## b. Dummy classifier

Build a dummy classifier that takes decisions randomly.

In [10]:
class DummyClassifier():
    
    def __init__(self):
        """
        Initialize the class.
        """
        pass
    
    def fit(self, X, y):
        """
        Fit the dummy classifier.
        
        Parameters
        ----------
        X : Numpy array or Pandas DataFrame of shape (n_samples, n_features)
            Training data.
        y : Numpy array or Pandas DataFrame of shape (n_samples,)
            Target values.
        """
        
        # no training required for random classifier
        
        pass
    
    def predict(self, X):
        """
        Predict the class labels for the provided data.

        Parameters
        ----------
        X : Numpy array or Pandas DataFrame of shape (n_queries, n_features)
            Test samples.

        Returns
        -------
        y : Numpy array or Pandas DataFrame of shape (n_queries,)
            Class labels for each data sample.
        """
        random_array = np.random.randint(2, size=(X.shape[0],1))
        
        return pd.DataFrame(random_array, columns = ['LoanStatus'])

Implement a function to evaluate the performance of a classification by computing the accuracy ($N_{correct}/N$).

In [11]:
def accuracy_score(y_true, y_pred):
    good_result = 0
    N = y_pred.shape[0]
    
    y_true_np = y_true.to_numpy()
    y_pred_np = y_pred.to_numpy()
    
    for i in range(0, N):
        if y_true_np[i] == y_pred_np[i]:
            good_result += 1
    
    return np.round(good_result/N, 4)

Compute the performance of the dummy classifier using the provided test set.

In [12]:
dummy_clf = DummyClassifier()

dummy_clf.fit(X_train, y_train)

y_pred = dummy_clf.predict(X_test)

print(f'Accuracy score : {accuracy_score(y_test, y_pred)}')
#print(y)



Accuracy score : 0.4583


## c. K-Nearest Neighbors classifier

Build a K-Nearest Neighbors classifier using an Euclidian distance computation and a simple majority voting criterion.

In [13]:
class KNNClassifier():
    
    def __init__(self, n_neighbors=3):
        """
        Initialize the class.
        
        Parameters
        ----------
        n_neighbors : int, default=3
            Number of neighbors to use by default.
        """
        self.n_neighbors = n_neighbors
        self.X = None
        self.y = None
    
    def fit(self, X, y):
        """
        Fit the k-nearest neighbors classifier.
        
        Parameters
        ----------
        X : Numpy array or Pandas DataFrame of shape (n_samples, n_features)
            Training data.
        y : Numpy array or Pandas DataFrame of shape (n_samples,)
            Target values.
        """
        self.X_train = X.to_numpy()
        self.y_train = y.to_numpy()
    
    @staticmethod
    def _euclidian_distance(a, b):
        """
        Utility function to compute the euclidian distance.
        
        Parameters
        ----------
        a : Numpy array or Pandas DataFrame
            First operand.
        b : Numpy array or Pandas DataFrame
            Second operand.
        """
        return np.sqrt(np.sum(np.square(a-b)))
    
    def predict(self, X):
        """
        Predict the class labels for the provided data.

        Parameters
        ----------
        X : Numpy array or Pandas DataFrame of shape (n_queries, n_features)
            Test samples.

        Returns
        -------
        y : Numpy array or Pandas DataFrame of shape (n_queries,)
            Class labels for each data sample.
        """
        
        X_np = X.to_numpy()
        y_pred = np.zeros(X_np.shape[0])
        
        for row in range(0, X_np.shape[0]):
            
            distances = np.zeros(self.X_train.shape[0])
            X_row = X_np[row]
            
            # compute distances
            for j in range(0, self.X_train.shape[0]):
                distances[j] = self._euclidian_distance(self.X_train[j], X_row)
        
            # get the k nearest neighbors
            nearest_neighbor_ids = distances.argsort()[:self.n_neighbors]
            # print(nearest_neighbor_ids)
        
            # choose the right answer
            nearest_neighbor_rings = self.y_train[nearest_neighbor_ids]
            # print(nearest_neighbor_rings)
            
            prediction = nearest_neighbor_rings.mean().round()
            # print(prediction)
            y_pred[row] = prediction
            
        
        return pd.DataFrame(y_pred, columns = ['LoanStatus'])

Compute the performance of the system as a function of $k = 1...7$.

In [14]:
for k in range(1,8):
    knn = KNNClassifier(k)
    

    # train algorythm
    knn.fit(X_train, y_train)
    
    # test algorythm
    y_pred = knn.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f'K = {k} : accuracy = {accuracy}')

K = 1 : accuracy = 0.6979
K = 2 : accuracy = 0.6354
K = 3 : accuracy = 0.7917
K = 4 : accuracy = 0.7396
K = 5 : accuracy = 0.8125
K = 6 : accuracy = 0.7812
K = 7 : accuracy = 0.8021


### What value of k gives you the best performances ? Comment your result
The best K is 5 with an accuracy on the test set of 81,25%. That's a good accuracy. Because the case is very linear.

Run the KNN algorithm using only the features `TotalIncome` and `CreditHistory`.

In [15]:
X_train_modif = X_train[['TotalIncome','CreditHistory']]
X_test_modif = X_test[['TotalIncome','CreditHistory']]

knn = KNNClassifier()


# train algorythm
knn.fit(X_train_modif, y_train)

# test algorythm
y_pred = knn.predict(X_test_modif)

accuracy = accuracy_score(y_test, y_pred)

print(f'only the features TotalIncome and CreditHistory : accuracy = {accuracy}')

knn = KNNClassifier()


# train algorythm
knn.fit(X_train, y_train)

# test algorythm
y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f'normal : accuracy = {accuracy}')

only the features TotalIncome and CreditHistory : accuracy = 0.7812
normal : accuracy = 0.7917


The performance with all the features is better than with only TotalIncome and CreditHistory

Re-run the KNN algorithm using the features `TotalIncome`, `CreditHistory` and `Married`.

In [16]:
X_train_modif = X_train[['TotalIncome', 'CreditHistory', 'Married']]
X_test_modif = X_test[['TotalIncome', 'CreditHistory', 'Married']]

knn = KNNClassifier()


# train algorythm
knn.fit(X_train_modif, y_train)

# test algorythm
y_pred = knn.predict(X_test_modif)

accuracy = accuracy_score(y_test, y_pred)

print(f'only the features TotalIncome and CreditHistory : accuracy = {accuracy}')



only the features TotalIncome and CreditHistory : accuracy = 0.8646


The performance with all the features is worst than with only TotalIncome and CreditHistory and married. That's the higher accuracy settings we tried.

Re-run the KNN algorithm using all features.

In [17]:
knn = KNNClassifier()


# train algorythm
knn.fit(X_train, y_train)

# test algorythm
y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f'normal : accuracy = {accuracy}')

normal : accuracy = 0.7917


Using the good combination of parameters (not all) give the best results. There is not many data for training the system. So it can be worst to have a lot of features. For this case, the best parameters are TotalIncome, CreditHistory and Married. But with a bigger dataset it can change.

In [18]:
print(np.round(0.5))

0.0


### How is your system taking decisions when you have an equal number of votes for both classes with values of k = 2, 4, 6 ?
In the example above, we see that the program favorite the 0 (no loan), when we have an equal number of vote.